In [1]:
#h2 heding

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import datetime
import calendar
%matplotlib inline

In [105]:
mytime = datetime.strptime('2014-12-01 00:00:00', '%Y-%m-%d %H:%M:%S')
mytime

datetime.datetime(2014, 12, 1, 0, 0)

In [106]:
rng = pd.date_range('2014-12-01',periods=35040 ,freq="15min")
lmn = range(0,35040)
print (len(rng))

35040


In [107]:
def timer(f):
    start = time.time()
    res=f()
    end =time.time()
    print("fitting time {}".format(end-start))
    return res

In [108]:
frame = pd.DataFrame()
newcsvData =[]
for i in range(1,11):
    filePath = 'Year1\\Home' + str(i) + '_yr1.csv'
    data1 = pd.read_csv(filePath , usecols=[0], names=['electricUsage'], header=None)
    data1['userId'] = i
    data1['dateVal'] =lmn
    data1['month'] = rng.month
    data1['dayOfWeek'] = rng.weekday_name
    data1['weekday'] = rng.weekday
    data1['hour']=rng.hour
    data1['winter']=np.where(((data1.month==12)|(data1.month==11)|(data1.month==10)|(data1.month==4)|(data1.month==3)|(data1.month==2)|(data1.month==1)),0,1)
    data1['winter']=data1['winter'].astype('int')
    dummy1=pd.get_dummies(data1['dayOfWeek'])
    data1=data1.merge(dummy1,left_index=True,right_index=True)
    dummy2=pd.get_dummies(data1['month'])
    data1=data1.merge(dummy2,left_index=True,right_index=True)
    newcsvData.append(data1)
    #print(newcsvData)
frame=pd.concat(newcsvData)    

In [109]:
def getUserData(id, frame):
    currentdata = frame[frame.userId==id]
    currentdata = currentdata.sample(frac=1).reset_index(drop=True)
    return   np.split(currentdata, [int(.6*len(currentdata)), int(.8*len(currentdata))])

In [110]:
frame.head()
#frame.isnull().sum()

,electricUsage,userId,dateVal,month,dayOfWeek,weekday,hour,winter,Friday,Monday,...,3,4,5,6,7,8,9,10,11,12
0,0.65018,1,0,12,Monday,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1.45400,1,1,12,Monday,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0.72971,1,2,12,Monday,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,3.10750,1,3,12,Monday,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0.63572,1,4,12,Monday,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [111]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
from sklearn.linear_model import Ridge
import xgboost as xgb1
from sklearn.ensemble import RandomForestRegressor

In [112]:
regr = LinearRegression()
ridge = Ridge()
xgb = xgb1.XGBRegressor(n_estimator=100,learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1,max_depth=7)
classifiers = [
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.PassiveAggressiveRegressor(),
]
rf = RandomForestRegressor()
maeLR =[]
maeRidge=[]
maeXgb=[]
maeRf=[]

In [113]:
for i in range(1, 11):
    #print ("For user : ", i)
    train, validate, test = getUserData(i,frame)
    y_train = train.electricUsage.to_frame()
    x_train = train.drop(['electricUsage','userId','month','weekday','dateVal','dayOfWeek','winter'],axis=1)
    y_test = test.electricUsage.to_frame()
    x_test = test.drop(['electricUsage','userId','month','weekday','dateVal','dayOfWeek','winter'],axis=1)
    
    #Linera Regression
    regr.fit(x_train, y_train)
    y_pred = regr.predict(x_test)
    #print("rms linear Regression " ,mean_squared_error(y_test, y_pred))
    #print("mean absolute for LR" ,mean_absolute_error(y_test, y_pred))
    maeLR.append(mean_absolute_error(y_test, y_pred))
    
    #XGB CLASSIFIER 
    xgb.fit(x_train, y_train)
    y_predXGB =xgb.predict(x_test)
    #print("mean Error XGB ",mean_squared_error(y_test, y_predXGB))
    #print("mean absolute for xgb" ,mean_absolute_error(y_test, y_predXGB))
    maeXgb.append(mean_absolute_error(y_test, y_predXGB))
    
    #Random Forest
    rf.fit(x_train, y_train)
    ypredRf = rf.predict(x_test)
    #print("mean Error rf " ,mean_squared_error(y_test, ypredRf))
    #print("mean absolute for rf ",mean_absolute_error(y_test, ypredRf))
    maeRf.append(mean_absolute_error(y_test, ypredRf))
    
    #Lineear Ridge 
    ridge.fit(x_train, y_train)
    ypredRidge = ridge.predict(x_test)
    #print("mean Error ridge " ,mean_squared_error(y_test, ypredRidge))
    #print("mean absolute for ridge ",mean_absolute_error(y_test, ypredRidge))
    maeRidge.append(mean_absolute_error(y_test, ypredRidge))

C:\Users\kelad\Anaconda2\lib\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [114]:
mse = pd.DataFrame(
    {'LinearRegression': maeLR,
     'RidgeExpression': maeRidge,
     'XGB': maeXgb,
     'RandomForest': maeRf,
    })
print(mse)

   LinearRegression  RandomForest  RidgeExpression       XGB
0          0.773981      0.730451         0.774013  0.710590
1          0.872683      0.877362         0.872735  0.849566
2          0.718097      0.698572         0.718167  0.677758
3          0.650748      0.607964         0.650788  0.591269
4          0.841604      0.825088         0.841677  0.798556
5          0.775506      0.768318         0.775535  0.744627
6          0.888061      0.854605         0.888121  0.824806
7          1.001448      0.983688         1.001534  0.958172
8          0.567067      0.571377         0.567117  0.548501
9          0.664078      0.675426         0.664193  0.653530
